In [1]:
# List of imports.
import os
import os.path
import glob

import numpy as np
from PIL import Image

from sklearn.preprocessing import LabelEncoder

In [40]:
from keras.utils import np_utils

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications import VGG16

from keras.models import Sequential,Model
from keras.layers import Input,Flatten,Dense,Dropout,GlobalAveragePooling2D,Conv2D,MaxPooling2D

In [3]:
imagedir = "Datasets/malimg_paper_dataset_imgs"

cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

In [4]:
# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %20s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label
    
# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in image.list_pictures(list_fams[i], ext='jpg|jpeg|bmp|png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        X[cnt] = x
        #x /= 255
        #x = preprocess_input(x)
        #X[cnt] = np.array(x).reshape(width*height*channels)
        cnt += 1
print("Images processed: %d" %(cnt))

Label: 0	Family:            Adialer.C	Number of images: 80
Label: 1	Family:            Agent.FYI	Number of images: 80
Label: 2	Family:            Allaple.A	Number of images: 80
Label: 3	Family:            Allaple.L	Number of images: 80
Label: 4	Family:        Alueron.gen!J	Number of images: 80
Label: 5	Family:            Autorun.K	Number of images: 80
Label: 6	Family:          C2LOP.gen!g	Number of images: 80
Label: 7	Family:              C2LOP.P	Number of images: 80
Label: 8	Family:       Dialplatform.B	Number of images: 80
Label: 9	Family:            Dontovo.A	Number of images: 80
Label:10	Family:             Fakerean	Number of images: 80
Label:11	Family:        Instantaccess	Number of images: 80
Label:12	Family:           Lolyda.AA1	Number of images: 80
Label:13	Family:           Lolyda.AA2	Number of images: 80
Label:14	Family:           Lolyda.AA3	Number of images: 80
Label:15	Family:            Lolyda.AT	Number of images: 80
Label:16	Family:          Malex.gen!J	Number of images: 

In [5]:
os.chdir(cur_dir)

In [6]:
X

array([[[[  90.,   90.,   90.],
         [   0.,    0.,    0.],
         [   0.,    0.,    0.],
         ..., 
         [   1.,    1.,    1.],
         [   0.,    0.,    0.],
         [   0.,    0.,    0.]],

        [[   0.,    0.,    0.],
         [   0.,    0.,    0.],
         [   0.,    0.,    0.],
         ..., 
         [   0.,    0.,    0.],
         [   0.,    0.,    0.],
         [   0.,    0.,    0.]],

        [[   0.,    0.,    0.],
         [   0.,    0.,    0.],
         [   0.,    0.,    0.],
         ..., 
         [   0.,    0.,    0.],
         [   0.,    0.,    0.],
         [   0.,    0.,    0.]],

        ..., 
        [[ 187.,  187.,  187.],
         [ 202.,  202.,  202.],
         [ 171.,  171.,  171.],
         ..., 
         [ 202.,  202.,  202.],
         [  19.,   19.,   19.],
         [   4.,    4.,    4.]],

        [[  90.,   90.,   90.],
         [  80.,   80.,   80.],
         [  21.,   21.,   21.],
         ..., 
         [  41.,   41.,   41.],
       

In [7]:
X.shape

(2000, 224, 224, 3)

In [8]:
encoder = LabelEncoder()
encoder.fit(y)
y_encoded = encoder.transform(y)
Y = np_utils.to_categorical(y_encoded)

In [41]:
image_shape = (224, 224, 3)

base_model = VGG16(weights='imagenet', input_shape=image_shape, include_top=False)
for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(160, activation='relu', name='fc1'))
model.add(Dropout(0.2))
model.add(Dense(160, activation='relu', name='fc2'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax', name='predictions'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X, Y, epochs=1, batch_size=num_samples, verbose=1)

Epoch 1/1


In [ ]:
preds

In [ ]:
preds.shape